Concurrent ANOVA Test

In [1]:
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import os

output_dir = '/content/drive/MyDrive/Projects/ds_N-Hari-Sai-Vignesh/outputs'

def run_anova_tests(file_path='merged_daily_data.csv'):
    """
    Performs ANOVA tests to check for significant differences in trading
    metrics across different market sentiment classifications.
    """

    try:

        df = pd.read_csv(file_path, parse_dates=['date'], index_col='date')


        unique_sentiments = df['classification'].unique()
        print(f"\nFound {len(unique_sentiments)} sentiment groups to compare: {unique_sentiments}")

        # Test 1: Total PnL (total_pnl_usd)
        print("\nTest 1: total_pnl_usd -")
        pnl_groups = [df[df['classification'] == sentiment]['total_pnl_usd'] for sentiment in unique_sentiments]

        # Run the ANOVA test
        f_statistic_pnl, p_value_pnl = stats.f_oneway(*pnl_groups)

        print(f"  F-Statistic: {f_statistic_pnl:.4f}")
        print(f"  P-Value: {p_value_pnl:.4f}")

        # Results
        if p_value_pnl < 0.05:
            print("  Result: SIGNIFICANT. The average PnL is statistically different")
            print("  between at least two of the sentiment groups.")
        else:
            print("  Result: NOT SIGNIFICANT. We cannot conclude the average PnL is")
            print("  different between the sentiment groups.")


        # Test 2: Total Volume (total_volume_usd)
        print("\nTest 2: total_volume_usd -")
        volume_groups = [df[df['classification'] == sentiment]['total_volume_usd'] for sentiment in unique_sentiments]

        # Run the ANOVA test
        f_statistic_vol, p_value_vol = stats.f_oneway(*volume_groups)

        print(f"  F-Statistic: {f_statistic_vol:.4f}")
        print(f"  P-Value: {p_value_vol:.4f}")

        # Results
        if p_value_vol < 0.05:
            print("  Result: SIGNIFICANT. The average Volume is statistically different")
            print("  between at least two of the sentiment groups.")
        else:
            print("  Result: NOT SIGNIFICANT. We cannot conclude the average Volume is")
            print("  different between the sentiment groups.")


        # Test 3: PnL Volatility (pnl_volatility)
        print("\nTest 3: pnl_volatility -")
        volatility_groups = [df[df['classification'] == sentiment]['pnl_volatility'] for sentiment in unique_sentiments]

        # Run the ANOVA test
        f_statistic_vola, p_value_vola = stats.f_oneway(*volatility_groups)

        print(f"  F-Statistic: {f_statistic_vola:.4f}")
        print(f"  P-Value: {p_value_vola:.4f}")

        # Results
        if p_value_vola < 0.05:
            print("  Result: SIGNIFICANT. The average PnL Volatility is statistically different")
            print("  between at least two of the sentiment groups.")
        else:
            print("  Result: NOT SIGNIFICANT. We cannot conclude the average PnL Volatility is")
            print("  different between the sentiment groups.")

        # Post-Hoc Test (Tukey's HSD)
        # This is only useful if an ANOVA test was significant.
        # It tells us *which specific groups* are different from each other.

        if p_value_pnl < 0.05:
            print("\nPost-Hoc Test for total_pnl_usd (Tukey's HSD) - ")
            print("This test shows which specific pairs are different:\n")
            tukey_results = pairwise_tukeyhsd(endog=df['total_pnl_usd'],
                                              groups=df['classification'],
                                              alpha=0.05)
            print(tukey_results)

            # Save results to a file
            os.makedirs(output_dir, exist_ok=True)
            with open(f"{output_dir}/tukey_pnl_results.txt", "w") as f:
                f.write(str(tukey_results))

    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Run the analysis
run_anova_tests('/content/drive/MyDrive/Projects/ds_N-Hari-Sai-Vignesh/csv_files/merged_daily_data.csv')


Found 5 sentiment groups to compare: ['Fear' 'Neutral' 'Greed' 'Extreme Greed' 'Extreme Fear']

Test 1: total_pnl_usd -
  F-Statistic: 1.2370
  P-Value: 0.2978
  Result: NOT SIGNIFICANT. We cannot conclude the average PnL is
  different between the sentiment groups.

Test 2: total_volume_usd -
  F-Statistic: 1.9933
  P-Value: 0.0985
  Result: NOT SIGNIFICANT. We cannot conclude the average Volume is
  different between the sentiment groups.

Test 3: pnl_volatility -
  F-Statistic: 0.6685
  P-Value: 0.6148
  Result: NOT SIGNIFICANT. We cannot conclude the average PnL Volatility is
  different between the sentiment groups.


In [3]:
data = '/content/drive/MyDrive/Projects/ds_N-Hari-Sai-Vignesh/csv_files/merged_daily_data.csv'

Lagged ANOVA Test

In [5]:
import pandas as pd
import scipy.stats as stats
import os

def perform_lagged_anova(file_path='merged_daily_data.csv'):
    """
    Performs lag analysis (ANOVA) to see if yesterday's sentiment
    predicts today's trading behavior (PnL and Volume).
    """
    try:

        df = pd.read_csv(file_path, parse_dates=['date'], index_col='date')
        df.sort_index(inplace=True)

        # Create Lagged Feature
        # Create a 1-day lag for both the numeric value and the classification
        df['value_lag_1'] = df['value'].shift(1)
        df['classification_lag_1'] = df['classification'].shift(1)

        # Clean Data
        # Lagging introduces NaN values for the first row
        original_rows = df.shape[0]
        df.dropna(subset=['classification_lag_1', 'value_lag_1'], inplace=True)
        print(f"Dropped {original_rows - df.shape[0]} row with NaN values after lagging.")

        if df.empty:
            print("Error: No data left after dropping NaNs.")
            return

        # Run ANOVA on Lagged Classification
        # Get unique classifications for the lagged data
        unique_sentiments_lag1 = df['classification_lag_1'].unique()

        if len(unique_sentiments_lag1) < 2:
            print("Skipping ANOVA for lag 1: Not enough unique sentiment groups.")
            return

        print(f"Comparing {len(unique_sentiments_lag1)} groups from yesterday's sentiment: {unique_sentiments_lag1}")

        # Test 1: Yesterday's Sentiment vs. Today's PnL
        print("\nTest: classification_lag_1 vs. total_pnl_usd")
        pnl_groups_lag1 = [df[df['classification_lag_1'] == sentiment]['total_pnl_usd']
                           for sentiment in unique_sentiments_lag1]

        # Run the ANOVA test for PnL
        f_stat_pnl_lag1, p_value_pnl_lag1 = stats.f_oneway(*pnl_groups_lag1)

        print(f"  F-Statistic: {f_stat_pnl_lag1:.4f}")
        print(f"  P-Value: {p_value_pnl_lag1:.4f}")

        if p_value_pnl_lag1 < 0.05:
            print("  Result: SIGNIFICANT! Yesterday's sentiment has a statistical")
            print("  relationship with today's total PnL.")
        else:
            print("  Result: NOT SIGNIFICANT. Yesterday's sentiment does not")
            print("  statistically predict today's total PnL.")

        # Test 2: Yesterday's Sentiment vs. Today's Volume
        print("\nTest: classification_lag_1 vs. total_volume_usd")
        vol_groups_lag1 = [df[df['classification_lag_1'] == sentiment]['total_volume_usd']
                           for sentiment in unique_sentiments_lag1]

        # Run the ANOVA test for Volume
        f_stat_vol_lag1, p_value_vol_lag1 = stats.f_oneway(*vol_groups_lag1)

        print(f"  F-Statistic: {f_stat_vol_lag1:.4f}")
        print(f"  P-Value: {p_value_vol_lag1:.4f}")

        if p_value_vol_lag1 < 0.05:
            print("  Result: SIGNIFICANT! Yesterday's sentiment has a statistical")
            print("  relationship with today's total volume.")
        else:
            print("  Result: NOT SIGNIFICANT. Yesterday's sentiment does not")
            print("  statistically predict today's total volume.")

    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        print("Please make sure 'merged_daily_data.csv' is in the same folder.")
    except ImportError:
        print("Error: Required library not found.")
        print("Please run: pip install pandas scipy")
    except Exception as e:
        print(f"An error occurred: {e}")

# Run the lag analysis
perform_lagged_anova(data)

Dropped 1 row with NaN values after lagging.
Comparing 5 groups from yesterday's sentiment: ['Fear' 'Neutral' 'Greed' 'Extreme Greed' 'Extreme Fear']

Test: classification_lag_1 vs. total_pnl_usd
  F-Statistic: 0.9636
  P-Value: 0.4294
  Result: NOT SIGNIFICANT. Yesterday's sentiment does not
  statistically predict today's total PnL.

Test: classification_lag_1 vs. total_volume_usd
  F-Statistic: 3.0853
  P-Value: 0.0179
  Result: SIGNIFICANT! Yesterday's sentiment has a statistical
  relationship with today's total volume.


Tukey's HSD Post-Hoc Test

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

def run_tukey_posthoc_test(file_path='merged_daily_data.csv'):
    """
    Performs a Tukey's HSD post-hoc test to identify which specific
    lagged sentiment groups have a significant effect on today's total volume.
    """
    try:
        df = pd.read_csv(file_path, parse_dates=['date'], index_col='date')

        df.sort_index(inplace=True)

        # Create the 1-day lagged classification feature
        df['classification_lag_1'] = df['classification'].shift(1)
        df.dropna(subset=['classification_lag_1'], inplace=True)

        # Tukey's HSD Test
        # This is the core statistical test
        tukey_results = pairwise_tukeyhsd(
            endog=df['total_volume_usd'],
            groups=df['classification_lag_1'],
            alpha=0.05
        )

        print("\n--- Tukey's HSD Test Results ---")
        print(tukey_results)

        # Save Test Results
        results_path = f"{output_dir}/tukey_volume_lag1_results.txt"

        with open(results_path, "w") as f:
            f.write("Tukey's HSD Post-Hoc Test Results\n")
            f.write("Variable: total_volume_usd\n")
            f.write("Group: classification_lag_1 (Yesterday's Sentiment)\n\n")
            f.write(str(tukey_results))

        # Visualize the Difference

        # Get the logical order of sentiments for plotting
        all_possible_sentiments = ['Extreme Fear', 'Fear', 'Neutral', 'Greed', 'Extreme Greed']
        unique_classifications = df['classification_lag_1'].unique()
        sentiment_order = [s for s in all_possible_sentiments if s in unique_classifications]

        plt.figure(figsize=(12, 7))
        sns.boxplot(
            data=df,
            x='classification_lag_1',
            y='total_volume_usd',
            order=sentiment_order
        )
        plt.title("Today's Total Volume vs. Yesterday's Sentiment")
        plt.xlabel("Yesterday's Sentiment Classification")
        plt.ylabel("Today's Total Volume (USD)")
        plt.tight_layout()

        #Save the results
        plot_path = f"{output_dir}/12_volume_by_lagged_sentiment_boxplot.png"
        plt.savefig(plot_path)
        plt.clf()
        plt.close()

    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        print("Please make sure 'merged_daily_data.csv' is in the same folder.")
    except ImportError:
        print("Error: Required library not found.")
        print("Please run: pip install pandas statsmodels matplotlib seaborn")
    except Exception as e:
        print(f"An error occurred: {e}")

# Run the post-hoc test
run_tukey_posthoc_test(data)


--- Tukey's HSD Test Results ---
               Multiple Comparison of Means - Tukey HSD, FWER=0.05                
    group1        group2       meandiff   p-adj      lower        upper     reject
----------------------------------------------------------------------------------
 Extreme Fear Extreme Greed -3774591.3107 0.0217 -7179866.2682 -369316.3532   True
 Extreme Fear          Fear -2362042.5331 0.3225 -5797373.4569 1073288.3908  False
 Extreme Fear         Greed -3265800.5631 0.0602 -6618340.2623    86739.136  False
 Extreme Fear       Neutral -3349646.1713  0.106 -7112862.6503  413570.3076  False
Extreme Greed          Fear  1412548.7777 0.1883  -366848.7918 3191946.3471  False
Extreme Greed         Greed   508790.7476 0.9072 -1104987.3005 2122568.7957  False
Extreme Greed       Neutral   424945.1394 0.9873 -1925917.5579 2775807.8367  False
         Fear         Greed  -903758.0301 0.5712 -2580027.8797  772511.8195  False
         Fear       Neutral  -987603.6383 0.7854 -338